In [1]:
import openai
import os
import numpy as np
from dotenv import load_dotenv
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import VectorRetriever
import torch

In [2]:
# LLM을 사용한 수학 키워드 및 대분류 추출 함수
def extract_keywords_and_category_from_math_problem(text):
    prompt = (
        "다음 수학 문제에서 수학과 관련된 키워드와 가장 적합한 대분류를 추출하세요. "
        "키워드는 '키워드:'로 시작하고 대분류는 '대분류:'로 시작하여 각각 추출해주세요. "
        "대분류는 '변화와 관계', '도형과 측정', '자료와 가능성' 중에서 하나를 먼저 확인하고, "
        "그래프나 표와 관련된 문제는 우선적으로 '자료와 가능성'으로 분류하고, "
        "길이, 넓이, 들이, 무게 등의 단위 관련 문제는 '도형과 측정'으로 분류하며, "
        "배열, 규칙, 비율과 관련된 문제는 '변화와 관계'로 분류하세요. "
        "어디에도 속하지 않으면 '수와 연산'으로 분류하세요.\n"
        f"문제: {text}"
    )
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150,
        temperature=0.5,
    )
    return response.choices[0].message["content"].strip()


# 키워드 전처리 함수
def preprocess_keywords(raw_keywords):
    # 키워드 부분만 추출
    keywords = raw_keywords.split("키워드:")[1].strip().split(", ")
    # 각 키워드를 소문자로 변환하여 리스트로 반환
    return [kw.lower() for kw in keywords]

In [3]:
# 시드 고정
def set_seed(seed=12345):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# 시드 고정 설정
set_seed()

c:\Users\seong\anaconda3\envs\kst_pp\lib\site-packages\torch\random.py:46: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: 지정된 모듈을 찾을 수 없습니다. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  return default_generator.manual_seed(seed)


In [4]:
def calculate_similarity(keywords, leaf):
    """
    키워드와 리프 간의 유사도를 계산하는 함수 (GraphRAG)
    :param keywords: 키워드 리스트
    :param leaf: 리프 노드 이름
    :return: 유사도 값
    """
    query_keywords = " ".join(keywords)
    query_text = f"Calculate similarity between {query_keywords} and {leaf}"

    # GraphRAG를 통해 유사도 계산
    response = rag.search(query_text=query_text, retriever_config={"top_k": 1})
    similarity_score = response.answer[0].get("similarity", 0.0)

    return similarity_score

In [5]:
def find_similar_nodes_from_graph(keywords, category, level, similarity_threshold=0.8):
    """
    특정 카테고리 내에서 레벨별로 키워드와 유사도가 높은 항목을 탐색.
    :param keywords: 키워드 리스트
    :param category: 대분류/중분류 이름
    :param level: 검색할 노드의 레벨 ("Subcategory", "Leaf" 등)
    :param similarity_threshold: 유사도 임계값
    :return: 유사도가 높은 노드 리스트
    """
    # GraphRAG를 사용하여 유사도 검색
    query_text = f"Find nodes related to {', '.join(keywords)} in category {category} at level {level}"
    response = rag.search(query_text=query_text, retriever_config={"top_k": 5})

    # 결과에서 유사도 높은 노드들을 필터링
    matched_nodes = [
        node for node in response.answer if node["similarity"] >= similarity_threshold
    ]

    return [node["name"] for node in matched_nodes]

In [6]:
def find_subcategories(keywords, category):
    """
    대분류(category)에서 가장 유사한 중분류 하나를 찾는다.
    :param keywords: 키워드 리스트
    :param category: 대분류 이름
    :return: 가장 유사한 중분류
    """
    best_match = find_similar_nodes_from_graph(keywords, category, level="Subcategory")
    if best_match:
        return best_match

    print(f"중분류를 찾을 수 없습니다. 대분류: {category}")
    return []

In [7]:
def find_leaf_nodes(keywords, category_name, similarity_threshold=0.8):
    # 중분류 찾기
    subcategories = find_subcategories(keywords, category_name)
    if not subcategories:
        print(f"중분류를 찾을 수 없습니다. 대분류: {category_name}")
        return []

    leaf_nodes = []
    for subcategory in subcategories:
        print(f"중분류 '{subcategory}'에서 하위 노드 탐색 시작")
        # 해당 중분류에 연결된 item 찾기
        items = find_similar_nodes_from_graph(
            keywords,
            subcategory,
            level="Item",
            similarity_threshold=similarity_threshold,
        )
        if not items:
            print(f"중분류 '{subcategory}'에 연결된 item이 없습니다.")
            continue
        print(f"중분류 '{subcategory}'에 연결된 item: {items}")

        for item in items:
            # 해당 item에 연결된 leaf 찾기
            query = """
            MATCH (i:Item {name: $item})-[:HAS_CHILD]->(ln:Leaf)
            RETURN ln.name AS leaf
            """
            result = session.run(query, item=item)
            leaf_candidates = [record["leaf"] for record in result]

            for leaf in leaf_candidates:
                # 최하위 노드(leaf) 유사도 검증
                leaf_similarity = calculate_similarity(keywords, leaf)
                if leaf_similarity >= similarity_threshold:
                    print(
                        f"최하위 노드 '{leaf}'는 유사도 {leaf_similarity:.2f}로 선택됨"
                    )
                    leaf_nodes.append(leaf)
                else:
                    print(
                        f"최하위 노드 '{leaf}'는 유사도 {leaf_similarity:.2f}로 선택되지 않음"
                    )

In [8]:
def verify_classification_with_llm(classification, keywords):
    prompt = (
        f"다음 키워드: {', '.join(keywords)} 가 분류 '{classification}'에 정확히 적합한지 평가해주세요. "
        f"해당 분류가 모든 {', '.join(keywords)} 와 직접적으로 연관되어야 합니다. 예를 들어, 다음과 같은 경우를 고려하세요:"
        "키워드 '덧셈'이 분류 '곱셈'에 해당하는 경우 '부적합'으로 응답해야 하며, "
        "키워드 '원뿔'이 분류 '직육면체'에 해당하는 경우 '부적합'으로 응답해야 합니다. "
        "적합하지 않다면 '부적합'으로, 적합하다면 '적합'으로만 대답해주세요. "
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.5,
    )
    result = response.choices[0].message["content"].strip()
    print(f"LLM 응답: {result}")  # 응답 메시지 출력 (디버깅용)
    return result

In [9]:
def process_math_problem(problem):
    # 1단계: 키워드 및 대분류 추출
    raw_keywords_and_category = extract_keywords_and_category_from_math_problem(problem)
    print(f"추출된 키워드 및 대분류 (원본): {raw_keywords_and_category}")

    # 키워드와 대분류 분리
    try:
        raw_keywords, most_similar_category = raw_keywords_and_category.split(
            "\n대분류: "
        )
    except ValueError:
        print("응답 형식이 맞지 않습니다. 기본값으로 '수와 연산'을 사용합니다.")
        raw_keywords = raw_keywords_and_category
        most_similar_category = "수와 연산"

    keywords = preprocess_keywords(raw_keywords)
    print(f"전처리된 키워드: {keywords}")
    print(f"가장 유사한 대분류: {most_similar_category}")

    # 2단계: 하위 노드 탐색
    leaf_nodes = find_leaf_nodes(
        keywords, most_similar_category, similarity_threshold=0.6
    )
    if not leaf_nodes:
        print("하위 노드를 찾을 수 없습니다. 프로세스를 종료합니다.")
        return

    print(f"찾은 하위 노드: {leaf_nodes}")

    # 3단계: 최종 검증
    verified_labels = []
    for leaf in leaf_nodes:
        verification = verify_classification_with_llm(leaf, keywords)
        if verification == "적합":
            verified_labels.append(leaf)

    if verified_labels:
        print(f"최종 라벨: {verified_labels}")
    else:
        print("적합한 라벨을 찾을 수 없습니다.")

In [10]:
# .env 파일 로드
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
INDEX_NAME = "index_name"

# Neo4j 데이터베이스 연결
driver = GraphDatabase.driver(uri, auth=(username, password))


# Custom OpenAI Embeddings
class CustomOpenAIEmbeddings:
    def __init__(self, api_key, model="text-embedding-ada-002"):
        openai.api_key = api_key
        self.model = model

    def embed_query(self, query):
        response = openai.Embedding.create(
            model=self.model,
            input=query,
        )
        return response["data"][0]["embedding"]


# Custom Embeddings 객체 생성
embedder = CustomOpenAIEmbeddings(api_key=openai_api_key)

# Retriever 객체 생성
retriever = VectorRetriever(driver, INDEX_NAME, embedder)

# LangChain LLM (예시로 ChatOllama 사용)
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3:8b")

# GraphRAG 초기화
rag = GraphRAG(retriever=retriever, llm=llm)

# 유사도 검색 예시
problem = "나영이네 농장에서는 매실을 $10000$ 개 수확하여 한 상자에 $1000$ 개씩 담았습니다. 매실을 담은 상자는 모두 몇 상자인지 구해 보세요."
response = rag.search(query_text=problem, retriever_config={"top_k": 5})
print(response.answer)

# Neo4j 드라이버 종료
driver.close()

C:\Users\seong\AppData\Local\Temp\ipykernel_20904\1082425131.py:37: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:8b")


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FCA11395E0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))